In [57]:
library(data.table)
library(zoo)
library(forecast)
library(dplyr)  
library(readr)
library(ggplot2)

In [58]:
train <- read_csv("./dataset/train.csv", col_types=list(
  Store = col_integer(),
  DayOfWeek= col_integer(),
  Date = col_date(),
  Sales = col_integer(),
  Customers = col_integer(),
  Open = col_integer(),
  Promo = col_integer(),
  StateHoliday = col_character(),
  SchoolHoliday = col_integer()))


by_Date_Gap <- by_Date[by_Date$Date %in% 
                         seq(as.Date("2014-06-30"),as.Date("2015-01-01"),by="day"),]
head(by_Date_Gap,3)

tail(by_Date_Gap,3)

all_stores <- unique(train$Store)
stores_reporting <- train$Store[train$Date == as.Date("2014-7-1")]
missing_stores <- all_stores[!(all_stores %in% stores_reporting)]
missing_stores

for (date in seq(as.Date("2014-7-2"),as.Date("2014-12-31"),by="day")) {
  stores_reporting <- train$Store[train$Date == date]
  missing_on_date <- all_stores[!(all_stores %in% stores_reporting)]
  if (length(setdiff(missing_on_date,missing_stores)) > 0) {
    cat("Date:",date," Difference in missing stores",setdiff(missing_on_date,missing_stores))
  } 
}

stores_reporting <- train$Store[train$Date == as.Date("2013-1-1")]
additional_missing_store <- all_stores[!(all_stores %in% stores_reporting)]
additional_missing_store

Date,NumOfStores
2014-06-30,1115
2014-07-01,935
2014-07-02,935


Date,NumOfStores
2014-12-30,935
2014-12-31,935
2015-01-01,1115


[1]   13   20   22   32   36   41   46   51   52   58   72   76   81   89   99
 [16]  100  108  113  115  127  129  132  136  137  139  144  145  149  155  159
 [31]  164  165  172  174  181  183  186  190  191  192  204  215  218  231  243
 [46]  258  263  275  277  279  283  284  287  288  298  306  317  342  345  348
 [61]  365  385  399  407  412  413  420  427  428  429  430  434  457  471  477
 [76]  485  490  492  498  500  501  512  514  518  522  534  539  540  542  547
 [91]  555  571  573  575  587  598  604  611  612  619  620  629  633  636  637
[106]  638  644  646  650  660  669  670  671  677  684  694  701  702  706  710
[121]  711  712  716  719  736  739  744  750  766  771  775  778  797  804  805
[136]  806  815  820  825  842  851  858  859  879  884  890  893  900  902  903
[151]  904  909  915  919  920  932  941  952  974  977  989 1000 1004 1009 1012
[166] 1019 1027 1038 1041 1049 1056 1065 1067 1080 1092 1094 1102 1104 1107 1109

[1] 988

In [59]:
date <- as.Date("2013-1-1")
day_of_week <- unique(train$DayOfWeek[train$Date == date])
sales <- as.numeric(names(which.max(table(train$Sales[train$Date == date]))))
customers <- as.numeric(names(which.max(table(train$Customers[train$Date == date]))))
open <- as.numeric(names(which.max(table(train$Open[train$Date == date]))))
promo <- as.numeric(names(which.max(table(train$Promo[train$Date == date]))))
state_holiday <- names(which.max(table(train$StateHoliday[train$Date == date])))
school_holiday <- as.numeric(names(which.max(table(train$SchoolHoliday[train$Date == date]))))

missing_row <- data.frame(Store = additional_missing_store,
                          DayOfWeek = day_of_week,
                          Date = date,
                          Sales = sales,
                          Customers = customers,
                          Open = open,
                          Promo = promo,
                          StateHoliday = state_holiday,
                          SchoolHoliday = school_holiday)
missing_row

train <- rbind(train,missing_row)

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
988,2,2013-01-01,0,0,0,0,a,1


In [60]:
gap <- seq(as.Date("2014-7-1"),as.Date("2014-12-31"),by="day")
n_missing <- length(gap)*length(missing_stores)
missing_df <- data.frame(Store = integer(n_missing),
                         DayOfWeek = integer(n_missing),
                         Date = rep(gap,length(missing_stores)),
                         Sales = integer(n_missing),
                         Customers = integer(n_missing),
                         Open = integer(n_missing),
                         Promo = integer(n_missing),
                         StateHoliday = character(n_missing),
                         SchoolHoliday = integer(n_missing),
                         logSales = numeric(n_missing),
                         stringsAsFactors=FALSE)

In [61]:
gap
n_missing
missing_df

[1] "2014-07-01" "2014-07-02" "2014-07-03" "2014-07-04" "2014-07-05"
  [6] "2014-07-06" "2014-07-07" "2014-07-08" "2014-07-09" "2014-07-10"
 [11] "2014-07-11" "2014-07-12" "2014-07-13" "2014-07-14" "2014-07-15"
 [16] "2014-07-16" "2014-07-17" "2014-07-18" "2014-07-19" "2014-07-20"
 [21] "2014-07-21" "2014-07-22" "2014-07-23" "2014-07-24" "2014-07-25"
 [26] "2014-07-26" "2014-07-27" "2014-07-28" "2014-07-29" "2014-07-30"
 [31] "2014-07-31" "2014-08-01" "2014-08-02" "2014-08-03" "2014-08-04"
 [36] "2014-08-05" "2014-08-06" "2014-08-07" "2014-08-08" "2014-08-09"
 [41] "2014-08-10" "2014-08-11" "2014-08-12" "2014-08-13" "2014-08-14"
 [46] "2014-08-15" "2014-08-16" "2014-08-17" "2014-08-18" "2014-08-19"
 [51] "2014-08-20" "2014-08-21" "2014-08-22" "2014-08-23" "2014-08-24"
 [56] "2014-08-25" "2014-08-26" "2014-08-27" "2014-08-28" "2014-08-29"
 [61] "2014-08-30" "2014-08-31" "2014-09-01" "2014-09-02" "2014-09-03"
 [66] "2014-09-04" "2014-09-05" "2014-09-06" "2014-09-07" "2014-09-08"
 [71] "2014-09-09" "2014-09-10" "2014-09-11" "2014-09-12" "2014-09-13"
 [76] "2014-09-14" "2014-09-15" "2014-09-16" "2014-09-17" "2014-09-18"
 [81] "2014-09-19" "2014-09-20" "2014-09-21" "2014-09-22" "2014-09-23"
 [86] "2014-09-24" "2014-09-25" "2014-09-26" "2014-09-27" "2014-09-28"
 [91] "2014-09-29" "2014-09-30" "2014-10-01" "2014-10-02" "2014-10-03"
 [96] "2014-10-04" "2014-10-05" "2014-10-06" "2014-10-07" "2014-10-08"
[101] "2014-10-09" "2014-10-10" "2014-10-11" "2014-10-12" "2014-10-13"
[106] "2014-10-14" "2014-10-15" "2014-10-16" "2014-10-17" "2014-10-18"
[111] "2014-10-19" "2014-10-20" "2014-10-21" "2014-10-22" "2014-10-23"
[116] "2014-10-24" "2014-10-25" "2014-10-26" "2014-10-27" "2014-10-28"
[121] "2014-10-29" "2014-10-30" "2014-10-31" "2014-11-01" "2014-11-02"
[126] "2014-11-03" "2014-11-04" "2014-11-05" "2014-11-06" "2014-11-07"
[131] "2014-11-08" "2014-11-09" "2014-11-10" "2014-11-11" "2014-11-12"
[136] "2014-11-13" "2014-11-14" "2014-11-15" "2014-11-16" "2014-11-17"
[141] "2014-11-18" "2014-11-19" "2014-11-20" "2014-11-21" "2014-11-22"
[146] "2014-11-23" "2014-11-24" "2014-11-25" "2014-11-26" "2014-11-27"
[151] "2014-11-28" "2014-11-29" "2014-11-30" "2014-12-01" "2014-12-02"
[156] "2014-12-03" "2014-12-04" "2014-12-05" "2014-12-06" "2014-12-07"
[161] "2014-12-08" "2014-12-09" "2014-12-10" "2014-12-11" "2014-12-12"
[166] "2014-12-13" "2014-12-14" "2014-12-15" "2014-12-16" "2014-12-17"
[171] "2014-12-18" "2014-12-19" "2014-12-20" "2014-12-21" "2014-12-22"
[176] "2014-12-23" "2014-12-24" "2014-12-25" "2014-12-26" "2014-12-27"
[181] "2014-12-28" "2014-12-29" "2014-12-30" "2014-12-31"

[1] 33120

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,logSales
0,0,2014-07-01,0,0,0,0,,0,0
0,0,2014-07-02,0,0,0,0,,0,0
0,0,2014-07-03,0,0,0,0,,0,0
0,0,2014-07-04,0,0,0,0,,0,0
0,0,2014-07-05,0,0,0,0,,0,0
0,0,2014-07-06,0,0,0,0,,0,0
0,0,2014-07-07,0,0,0,0,,0,0
0,0,2014-07-08,0,0,0,0,,0,0
0,0,2014-07-09,0,0,0,0,,0,0
0,0,2014-07-10,0,0,0,0,,0,0


In [62]:
for (date in gap) {
  missing_df$Store[missing_df$Date == date] <- missing_stores
  
  day_of_week <- unique(train$DayOfWeek[train$Date == date])
  missing_df$DayOfWeek[missing_df$Date == date] <- rep(day_of_week, length(missing_stores))
  
  missing_df$Sales[missing_df$Date == date] <- rep(NA, length(missing_stores))

  missing_df$Customers[missing_df$Date == date] <- rep(NA, length(missing_stores))
  
  open <- as.numeric(names(which.max(table(train$Open[train$Date == date]))))
  missing_df$Open[missing_df$Date == date] <- rep(open, length(missing_stores))
  
  promo <- as.numeric(names(which.max(table(train$Promo[train$Date == date]))))
  missing_df$Promo[missing_df$Date == date] <- rep(promo, length(missing_stores))

  state_holiday <- names(which.max(table(train$StateHoliday[train$Date == date])))
  missing_df$StateHoliday[missing_df$Date == date] <- rep(state_holiday, length(missing_stores))

  school_holiday <- as.numeric(names(which.max(table(train$SchoolHoliday[train$Date == date]))))
  missing_df$SchoolHoliday[missing_df$Date == date] <- rep(school_holiday, length(missing_stores))
  
#   missing_df$logSales[missing_df$Date == date] <- rep(NA, length(missing_stores))

}

head(missing_df)

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,logSales
13,2,2014-07-01,NA,NA,1,1,0,0,0
13,3,2014-07-02,NA,NA,1,1,0,0,0
13,4,2014-07-03,NA,NA,1,1,0,0,0
13,5,2014-07-04,NA,NA,1,1,0,0,0
13,6,2014-07-05,NA,NA,1,0,0,0,0
13,7,2014-07-06,NA,NA,0,0,0,0,0


In [63]:
train$logSales <- log(train$Sales+1)
train_filled_gap <- rbind(train,missing_df)
train_filled_gap <- train_filled_gap[order(train_filled_gap$Date),]
train_filled_gap 
train_filled_gap <- train_filled_gap %>% 
                      group_by(Store, DayOfWeek, Open, Promo) %>%
                      mutate(Sales = as.integer(ifelse(is.na(Sales), 
                                                       ifelse(Open == 0, 
                                                              0,
                                                              median(Sales, na.rm=T)), 
                                                       Sales))) %>%
                      mutate(Customers = as.integer(ifelse(is.na(Customers),
                                                           ifelse(Open == 0, 
                                                              0,
                                                              median(Customers, na.rm=T)),
                                                           Customers))) %>%
                      mutate(logSales = ifelse(is.na(logSales),
                                               ifelse(Open == 0,
                                                      0,
                                                      mean(logSales, na.rm=T)), 
                                               logSales))
train_filled_gap 

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,logSales
1,2,2013-01-01,0,0,0,0,a,1,0
2,2,2013-01-01,0,0,0,0,a,1,0
3,2,2013-01-01,0,0,0,0,a,1,0
4,2,2013-01-01,0,0,0,0,a,1,0
5,2,2013-01-01,0,0,0,0,a,1,0
6,2,2013-01-01,0,0,0,0,a,1,0
7,2,2013-01-01,0,0,0,0,a,1,0
8,2,2013-01-01,0,0,0,0,a,1,0
9,2,2013-01-01,0,0,0,0,a,1,0
10,2,2013-01-01,0,0,0,0,a,1,0


Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,logSales
1,2,2013-01-01,0,0,0,0,a,1,0
2,2,2013-01-01,0,0,0,0,a,1,0
3,2,2013-01-01,0,0,0,0,a,1,0
4,2,2013-01-01,0,0,0,0,a,1,0
5,2,2013-01-01,0,0,0,0,a,1,0
6,2,2013-01-01,0,0,0,0,a,1,0
7,2,2013-01-01,0,0,0,0,a,1,0
8,2,2013-01-01,0,0,0,0,a,1,0
9,2,2013-01-01,0,0,0,0,a,1,0
10,2,2013-01-01,0,0,0,0,a,1,0


In [64]:
anything_missed <- subset(train_filled_gap, is.na(Sales) | is.na(Customers) | is.na(logSales))
anything_missed

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,logSales


In [65]:
store13 <- subset(train_filled_gap, Store==13)
ggplot(store13, aes(Date,Sales)) + geom_line() +geom_smooth() + ggtitle("Revenue for Store 13 over time")

`geom_smooth()` using method = 'loess'


ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): 无法启动png()装置


plot without title

In [ ]:
by_Date <- train %>% group_by(Date) %>% summarise(NumOfStores=n())
# ggplot(by_Date, aes(Date,NumOfStores)) + geom_line()

In [ ]:
library(ggplot2)
test <- fread("./dataset/test.csv")
train <- fread("./dataset/train.csv")
store <- fread("./dataset/store.csv")

In [ ]:
hist(train$Sales, 50, main="Histogram of Sales")

In [ ]:
train[Sales != 0]

In [ ]:
hist(aggregate(train[Sales != 0]$Sales, by = list(train[Sales != 0]$Store), mean)$x, 100, main = "Mean sales per store when store was not closed")

In [ ]:
str(train)
by_Date <- train %>% group_by(Date) %>% summarise(NumStores=n())
ggplot(by_Date, aes(x=Date,y=NumStores,group = 1)) + geom_line()

In [ ]:
str(test)

In [ ]:
str(store)

In [ ]:
head(test)

In [ ]:
train[, Date := as.Date(Date)]
head(train)

In [ ]:
head(store)

In [ ]:
train <- train[order(Date)]
test <- test[order(Date)]
summary(train)
summary(test)
summary(store)
# head(train)

In [ ]:
test[is.na(test$Open), ]

In [ ]:
test$Open[test$Store == 622]

In [ ]:
test[is.na(test)] <- 1

In [ ]:
test$Open[test$Store == 622]

In [ ]:
by_Date <- train %>% group_by(Date) %>% summarise(NumStores=n())
ggplot(by_Date, aes(Date,NumStores)) + geom_line()